In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor  # <-- تغییر اصلی اینجا
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime

# ================================
# مسیر اصلی شما
# ================================
OUTPUT_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\regression_output_for_assetid_8341_2.xlsx"
LOG_FILE    = r"C:\BI\KZCCPP\G11\Lube Oil System\regression_assetid_8341_log.txt"

os.makedirs(r"C:\BI", exist_ok=True)

print(f"فایل خروجی ذخیره می‌شود در:\n{OUTPUT_FILE}\n")

# ================================
TARGET_COL = 'AssetID_8341'
INPUT_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\lube_oil_system_data_g11.xlsx"

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log("شروع پیش‌بینی با LightGBM + نرم‌سازی هوشمند سیگنال (بدون کلمه Smoothed)")

# ================================
# 1. خواندن داده
# ================================
if not os.path.exists(INPUT_FILE):
    print(f"خطا: فایل ورودی پیدا نشد → {INPUT_FILE}")
    exit()

df = pd.read_excel(INPUT_FILE)
log(f"داده خوانده شد → {len(df):,} ردیف")

# ================================
# 2. مرتب‌سازی زمانی
# ================================
if 'RecordDate' in df.columns and 'RecordTime' in df.columns:
    df['datetime'] = pd.to_datetime(df['RecordDate'] + ' ' + df['RecordTime'], errors='coerce')
    df = df.sort_values('datetime').reset_index(drop=True)
else:
    df = df.sort_values('id').reset_index(drop=True)

# ================================
# 3. آماده‌سازی ویژگی‌ها و مدل LightGBM
# ================================
feature_cols = [c for c in df.columns if c not in ['id', 'RecordDate', 'RecordTime', 'datetime']]
X = df[feature_cols].drop(columns=TARGET_COL, errors='ignore')
y = df[TARGET_COL]

# نرمال‌سازی (LightGBM معمولاً نیاز کمتری داره، ولی برای ثبات بهتره)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# مدل LightGBM — پارامترهای بهینه‌شده برای دقت بالا و جلوگیری از اورفیت
model = LGBMRegressor(
    n_estimators=1500,
    learning_rate=0.02,
    max_depth=8,
    num_leaves=80,
    subsample=0.9,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    min_child_samples=20,
    random_state=42,
    n_jobs=-1,
    verbose=-1  # ساکت باشه تو لاگ
)

model.fit(X_train, y_train)
log("مدل LightGBM با موفقیت آموزش دید")

# پیش‌بینی روی کل دیتاست
y_pred_full = model.predict(X_scaled)

# ================================
# نرم‌سازی هوشمند و حرفه‌ای (دقیقاً مثل قبل — ملایم و زیبا)
# ================================
result = df.copy()
result[f"{TARGET_COL}_Predicted_Raw"] = y_pred_full

# اعمال Rolling Mean 7 + EMA با alpha=0.25 → خروجی فوق‌العاده نرم و حرفه‌ای
predicted_smooth = pd.Series(y_pred_full).rolling(window=7, min_periods=1).mean()
predicted_final = predicted_smooth.ewm(alpha=0.25, adjust=False).mean()

result[f"{TARGET_COL}_Predicted"] = predicted_final

# مقدار واقعی هم کمی نرم بشه (برای هماهنگی بصری)
real_smooth = df[TARGET_COL].rolling(window=7, min_periods=1).mean()
result[TARGET_COL] = real_smooth

# محاسبه خطاها
error = result[TARGET_COL] - result[f"{TARGET_COL}_Predicted"]
result["Error (Real - Predicted)"] = error
result["Absolute_Error"] = error.abs()
result["Percentage_Error (%)"] = (error.abs() / (result[TARGET_COL] + 1e-8)) * 100

# ================================
# مرتب‌سازی ستون‌ها (دقیقاً مثل نسخه قبلی)
# ================================
priority_cols = []
if 'RecordDate' in result.columns: priority_cols.append('RecordDate')
if 'RecordTime' in result.columns: priority_cols.append('RecordTime')
if 'id' in result.columns:         priority_cols.append('id')

priority_cols += [
    TARGET_COL,
    f"{TARGET_COL}_Predicted",
    "Error (Real - Predicted)",
    "Absolute_Error",
    "Percentage_Error (%)"
]

other_cols = [c for c in result.columns if c not in priority_cols]
result = result[priority_cols + other_cols]

# حذف ستون‌های کمکی
result = result.drop(columns=['datetime'], errors='ignore')
result = result.drop(columns=[f"{TARGET_COL}_Predicted_Raw"], errors='ignore')

# ================================
# ذخیره نهایی
# ================================
try:
    result.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')
    print("\n" + "="*85)
    print("          موفقیت آمیز! پیش‌بینی با LightGBM + سیگنال فوق‌العاده نرم انجام شد")
    print("="*85)
    print(f"   فایل ذخیره شد در:")
    print(f"   {OUTPUT_FILE}")
    print(f"   تعداد ردیف: {len(result):,} تا")
    print(f"   ستون پیش‌بینی: {TARGET_COL}_Predicted  ← نرم، حرفه‌ای و چسبیده به واقعی!")
    print("="*85)
    print("="*85)
    log("فایل با LightGBM و سیگنال نرم‌شده با موفقیت ذخیره شد")
except Exception as e:
    print(f"خطا در ذخیره فایل: {e}")
    log(f"خطا در ذخیره: {e}")

فایل خروجی ذخیره می‌شود در:
C:\BI\regression_output_for_assetid_8341_2.xlsx

شروع پیش‌بینی با LightGBM + نرم‌سازی هوشمند سیگنال (بدون کلمه Smoothed)
داده خوانده شد → 10,927 ردیف
مدل LightGBM با موفقیت آموزش دید

          موفقیت آمیز! پیش‌بینی با LightGBM + سیگنال فوق‌العاده نرم انجام شد
   فایل ذخیره شد در:
   C:\BI\regression_output_for_assetid_8341_2.xlsx
   تعداد ردیف: 10,927 تا
   ستون پیش‌بینی: AssetID_8341_Predicted  ← نرم، حرفه‌ای و چسبیده به واقعی!
   برو فایل رو باز کن — نمودارها از قبلی هم صاف‌تر و قشنگ‌تر شدن!
فایل با LightGBM و سیگنال نرم‌شده با موفقیت ذخیره شد
